# Plan

# Load the libraries and the data

In [53]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [56]:
ratings = pd.read_csv("./GBB/bgg-15m-reviews.csv")
ratings

,Unnamed: 0,user,rating,comment,ID,name
0,0,Torsten,10.0,NaN,30549,Pandemic
1,1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
2,2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
3,3,Mike Mayer,10.0,NaN,30549,Pandemic
4,4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic
...,...,...,...,...,...,...
15823264,15823264,Fafhrd65,8.0,Turn based preview looks very promising. The g...,281515,Company of Heroes
15823265,15823265,PlatinumOh,8.0,KS,281515,Company of Heroes
15823266,15823266,BunkerBill,7.0,NaN,281515,Company of Heroes
15823267,15823267,Hattori Hanzo,6.0,NaN,281515,Company of Heroes


In [57]:
ratings.columns

Index(['Unnamed: 0', 'user', 'rating', 'comment', 'ID', 'name'], dtype='object')

In [58]:
ratings.drop(["Unnamed: 0", "comment"], axis=1, inplace=True)

In [59]:
ratings.head()

,user,rating,ID,name
0,Torsten,10.0,30549,Pandemic
1,mitnachtKAUBO-I,10.0,30549,Pandemic
2,avlawn,10.0,30549,Pandemic
3,Mike Mayer,10.0,30549,Pandemic
4,Mease19,10.0,30549,Pandemic


In [60]:
# giving the columns better names

ratings.rename(mapper={"user":"user_id", "ID":"game_id", "name":"title"}, axis=1, inplace=True)
ratings.head()

,user_id,rating,game_id,title
0,Torsten,10.0,30549,Pandemic
1,mitnachtKAUBO-I,10.0,30549,Pandemic
2,avlawn,10.0,30549,Pandemic
3,Mike Mayer,10.0,30549,Pandemic
4,Mease19,10.0,30549,Pandemic


In [61]:
ratings.columns

Index(['user_id', 'rating', 'game_id', 'title'], dtype='object')

In [62]:
info = ratings[["game_id","title"]]
info

,game_id,title
0,30549,Pandemic
1,30549,Pandemic
2,30549,Pandemic
3,30549,Pandemic
4,30549,Pandemic
...,...,...
15823264,281515,Company of Heroes
15823265,281515,Company of Heroes
15823266,281515,Company of Heroes
15823267,281515,Company of Heroes


In [63]:
info.drop_duplicates(inplace=True)
info

C:\Users\linco\AppData\Local\Temp\ipykernel_27232\999174464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info.drop_duplicates(inplace=True)


,game_id,title
0,30549,Pandemic
100,822,Carcassonne
200,13,Catan
300,68448,7 Wonders
400,36218,Dominion
...,...,...
15823119,246345,Ninja Rush
15823149,195623,Politricks: Dirty Card Game
15823179,235943,Doppel X
15823209,284862,Beasty Borders


In [64]:
ratings.game_id.nunique()

19330

# Exploratory Analysis

In [65]:
# number of users

ratings.user_id.nunique()

351048

In [66]:
# number of books
ratings.game_id.nunique()

19330

In [67]:
# number of ratings per user
userfreq = ratings[["user_id", "game_id"]].groupby("user_id").count().reset_index()
userfreq.columns = ["user_id","no_ratings"]
userfreq.sort_values(by="no_ratings")

,user_id,no_ratings
84277,Khaisor,1
204393,christophersisto,1
204392,christophermrau,1
204391,christophermorrison2,1
204389,christophergodfrey,1
...,...,...
66952,Hessu68,4602
342435,warta,4811
41655,Doel,4876
161600,TomVasel,4950


# Bahesian stats

In [68]:
gamestats = ratings.groupby("game_id")["rating"].agg(["count","mean"])
gamestats.head(10)

,count,mean
game_id,,
1,5089,7.630658
2,544,6.613411
3,14393,7.445493
4,337,6.612821
5,17838,7.344081
6,81,6.479630
7,3150,6.520788
8,197,6.147411
9,1358,6.467161


In [69]:
C=gamestats["count"].mean()
m=gamestats["mean"].mean()

In [70]:
def bayesian_avg(ratings):
    bayesian_avg = (C*m+ratings.sum()) / (C+ratings.count())
    return bayesian_avg

In [71]:
bayesavg_ratings = ratings.groupby("game_id")["rating"].agg(bayesian_avg).reset_index()
bayesavg_ratings.columns = ["game_id","bayesian_avg"]
bayesavg_ratings.head(5)

,game_id,bayesian_avg
0,1,7.459099
1,2,6.480725
2,3,7.388831
3,4,6.456785
4,5,7.302331


In [72]:
gamestats = gamestats.merge(bayesavg_ratings, on="game_id")
gamestats.head(10)

,game_id,count,mean,bayesian_avg
0,1,5089,7.630658,7.459099
1,2,544,6.613411,6.480725
2,3,14393,7.445493,7.388831
3,4,337,6.612821,6.456785
4,5,17838,7.344081,7.302331
5,6,81,6.479630,6.400389
6,7,3150,6.520788,6.494336
7,8,197,6.147411,6.344997
8,9,1358,6.467161,6.439100
9,10,7855,6.717890,6.687185


In [73]:
gamestats = gamestats.merge(info[["game_id","title"]], on="game_id")
gamestats.head(10)

,game_id,count,mean,bayesian_avg,title
0,1,5089,7.630658,7.459099,Die Macher
1,2,544,6.613411,6.480725,Dragonmaster
2,3,14393,7.445493,7.388831,Samurai
3,4,337,6.612821,6.456785,Tal der Könige
4,5,17838,7.344081,7.302331,Acquire
5,6,81,6.479630,6.400389,Mare Mediterraneum
6,7,3150,6.520788,6.494336,Cathedral
7,8,197,6.147411,6.344997,Lords of Creation
8,9,1358,6.467161,6.439100,El Caballero
9,10,7855,6.717890,6.687185,Elfenland


# Building the recommender system

## Dictionaries

In [78]:
# making all the titles lower case so it is easier to search

info["title"] = info["title"].apply(lambda x: x.lower())
info["title"]

C:\Users\linco\AppData\Local\Temp\ipykernel_27232\569374738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info["title"] = info["title"].apply(lambda x: x.lower())


0                              pandemic
100                         carcassonne
200                               catan
300                           7 wonders
400                            dominion
                       ...             
15823119                     ninja rush
15823149    politricks: dirty card game
15823179                       doppel x
15823209                 beasty borders
15823239              company of heroes
Name: title, Length: 19330, dtype: object

In [79]:
gametitles = dict(zip(info["game_id"], info["title"]))
gametitles

{30549: 'pandemic',
 822: 'carcassonne',
 13: 'catan',
 68448: '7 wonders',
 36218: 'dominion',
 9209: 'ticket to ride',
 178900: 'codenames',
 31260: 'agricola',
 3076: 'puerto rico',
 40692: 'small world',
 167791: 'terraforming mars',
 70323: 'king of tokyo',
 14996: 'ticket to ride: europe',
 148228: 'splendor',
 2651: 'power grid',
 173346: '7 wonders duel',
 129622: 'love letter',
 169786: 'scythe',
 39856: 'dixit',
 478: 'citadels',
 230802: 'azul',
 28143: 'race for the galaxy',
 110327: 'lords of waterdeep',
 84876: 'the castles of burgundy',
 34635: 'stone age',
 163412: 'patchwork',
 1927: 'munchkin',
 65244: 'forbidden island',
 12333: 'twilight struggle',
 161936: 'pandemic legacy: season 1',
 150376: 'dead of winter: a crossroads game',
 120677: 'terra mystica',
 174430: 'gloomhaven',
 11: 'bohnanza',
 98778: 'hanabi',
 15987: 'arkham horror',
 50: 'lost cities',
 10547: 'betrayal at house on the hill',
 41114: 'the resistance',
 54043: 'jaipur',
 131357: 'coup',
 147020:

## The matrix

In [80]:
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.
    
    Args:
        df: pandas dataframe
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        game_mapper: dict that maps game id's to game indices
        game_inv_mapper: dict that maps game indices to game id's
    """
    
    N = df["user_id"].nunique() # number of users
    M= df["game_id"].nunique() # number of games
    
    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
    game_mapper = dict(zip(np.unique(df["game_id"]), list(range(M))))
    
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
    game_inv_mapper = dict(zip(list(range(M)), np.unique(df["game_id"])))
    
    user_index = [user_mapper[i] for i in df["user_id"]]
    game_index = [game_mapper[i] for i in df["game_id"]]
    
    X = csr_matrix((df["rating"], (game_index, user_index)), shape=(M,N))
    
    return X, user_mapper, game_mapper, user_inv_mapper, game_inv_mapper

In [81]:
ratings.user_id.unique()

array(['Torsten', 'mitnachtKAUBO-I', 'avlawn', ..., 'rielnano',
       'Lost_Titan_Legacy', 'smedzz31'], dtype=object)

In [82]:
# encode the user ids
from sklearn.preprocessing import LabelEncoder

# Step 1. Instantiate the model (label encoding)
le = LabelEncoder() 
le.fit_transform(ratings['user_id'])

# Step 2. Fit the variable to the instatiated model
ratings['user_id'] = le.fit_transform(ratings['user_id'])
ratings.head(10)

,user_id,rating,game_id,title
0,162393,10.0,30549,Pandemic
1,282186,10.0,30549,Pandemic
2,188488,10.0,30549,Pandemic
3,104525,10.0,30549,Pandemic
4,102294,10.0,30549,Pandemic
5,202289,10.0,30549,Pandemic
6,259457,10.0,30549,Pandemic
7,34278,10.0,30549,Pandemic
8,237240,10.0,30549,Pandemic
9,199851,10.0,30549,Pandemic


In [83]:
ratings.user_id.nunique()

351049

In [84]:
ratings.tail(10)

,user_id,rating,game_id,title
15823259,94173,9.0,281515,Company of Heroes
15823260,180855,9.0,281515,Company of Heroes
15823261,320655,9.0,281515,Company of Heroes
15823262,61909,9.0,281515,Company of Heroes
15823263,129799,9.0,281515,Company of Heroes
15823264,50913,8.0,281515,Company of Heroes
15823265,124124,8.0,281515,Company of Heroes
15823266,22867,7.0,281515,Company of Heroes
15823267,65652,6.0,281515,Company of Heroes
15823268,132441,1.0,281515,Company of Heroes


In [85]:
ratings.sort_values(by=["rating"],ascending=True)

,user_id,rating,game_id,title
2439405,59027,1.401300e-45,42,Tigris & Euphrates
9924871,240149,1.000000e-30,5432,Chutes and Ladders
8654503,182310,1.000000e-04,3699,Killer Bunnies and the Quest for the Magic Carrot
15015812,182310,1.000000e-03,9782,Battle of the Bands: Encore Edition
15250333,147014,1.000000e-03,18041,Bunco Party
...,...,...,...,...
3726695,327193,1.000000e+01,237182,Root
3726694,288547,1.000000e+01,237182,Root
3726693,139590,1.000000e+01,237182,Root
3726691,331038,1.000000e+01,237182,Root


In [86]:
# number of ratings per user
userfreq = ratings[["user_id", "game_id"]].groupby("user_id").count().reset_index()
userfreq.columns = ["user_id","no_ratings"]
userfreq.sort_values(by="no_ratings")

,user_id,no_ratings
245030,245030,1
169471,169471,1
147351,147351,1
49107,49107,1
95683,95683,1
...,...,...
66952,66952,4602
342435,342435,4811
41655,41655,4876
161600,161600,4950


In [87]:
ratings = ratings.merge(userfreq, on="user_id")
ratings

,user_id,rating,game_id,title,no_ratings
0,162393,10.0,30549,Pandemic,1358
1,162393,10.0,68448,7 Wonders,1358
2,162393,10.0,178900,Codenames,1358
3,162393,10.0,31260,Agricola,1358
4,162393,10.0,148228,Splendor,1358
...,...,...,...,...,...
15823264,811,10.0,281515,Company of Heroes,1
15823265,146512,10.0,281515,Company of Heroes,1
15823266,307953,10.0,281515,Company of Heroes,1
15823267,94173,9.0,281515,Company of Heroes,1


In [88]:
# actually creating the matrix

X, user_mapper, game_mapper, user_inv_mapper, game_inv_mapper = create_X(ratings)

In [89]:
X

<19330x351049 sparse matrix of type '<class 'numpy.float64'>'
	with 15823269 stored elements in Compressed Sparse Row format>

## k-nearest neighbors

In [90]:
from sklearn.neighbors import NearestNeighbors

KNN = NearestNeighbors(n_neighbors=10, metric="cosine") # how to pick the metric????
KNN.fit(X)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [91]:
# use the mapper to find the index of the book we want to assess

gameindex = game_mapper[10]
gameindex

9

In [92]:
game_toassess = X[9]
game_toassess

<1x351049 sparse matrix of type '<class 'numpy.float64'>'
	with 7855 stored elements in Compressed Sparse Row format>

In [93]:
# find the nearest neighbors to this book
neig = KNN.kneighbors(game_toassess, return_distance=False)
neig

array([[  9, 673,  84,  80,  51, 109, 429, 406, 218, 311]], dtype=int64)

In [94]:
# use the inverse mapper to find the ids of the neighbours

neig_ids = []
for i in range(1,10):
    n = neig.item(i)
    neig_ids.append(game_inv_mapper[n])

In [95]:
neig_ids

[826, 93, 88, 54, 120, 503, 475, 256, 361]

In [96]:
# use the ids to find the titles

print("because you enjoyed playing:",gametitles[9])
for i in neig_ids:
    print(gametitles[i])

because you enjoyed playing: el caballero
cartagena
el grande
torres
tikal
hoity toity
through the desert
taj mahal
mississippi queen
hare & tortoise


## Make  recommender function

In [97]:
def rec_games(game_id, df, X, k, metric="cosine"):
    """
    Finds k-nearest neighbours for a given book id.
    
    Args:
        book_id: id of the book of interest
        X: user-item utility matrix
        k: number of similar book to retrieve
        metric: distance metric for kNN calculations
    
    Returns:
        list of k similar book ID's
    """
        
    neighbour_ids = []
        
    game_ind = game_mapper[game_id]
    game_to_assess = X[game_ind]
    KNN = NearestNeighbors(n_neighbors=k, metric=metric)
        
    KNN.fit(X)
        
    neighbour = KNN.kneighbors(game_to_assess, return_distance=False)
        
    for i in range(1,k):
        n = neighbour.item(i)
        neighbour_ids.append(game_inv_mapper[n])
            
    game_titles = dict(zip(df["game_id"], df["title"]))
    game_title = game_titles[game_id]
        
    print("Because you enjoyed playing:", game_title) 
    print("I think you would enjoy these:")
    for i in neighbour_ids:
        print(game_titles[i])

In [98]:
rec_games(10, info, X, k=10)

Because you enjoyed playing: elfenland
I think you would enjoy these:
cartagena
el grande
torres
tikal
hoity toity
through the desert
taj mahal
mississippi queen
hare & tortoise


In [100]:
info[info.title.str.contains('pandemic')]

,game_id,title
0,30549,pandemic
2893,161936,pandemic legacy: season 1
5362289,221107,pandemic legacy: season 2
6539377,150658,pandemic: the cure
6545946,198928,pandemic: iberia
7406011,192153,pandemic: reign of cthulhu
9929732,260428,pandemic: fall of rome
9936696,157789,pandemic: contagion
11432557,234671,pandemic: rising tide
12224758,280789,pandemic: rapid response


In [101]:
rec_games(30549, info, X, k=10)

Because you enjoyed playing: pandemic
I think you would enjoy these:
7 wonders
carcassonne
dominion
catan
ticket to ride
codenames
small world
king of tokyo
love letter


In [102]:
# nemesis is the best game
info[info.title.str.contains('nemesis')]

,game_id,title
6538181,167355,nemesis
15467076,197076,nemesis: burma 1944
15671818,310100,nemesis: lockdown
15780467,200959,1813: napoleon's nemesis


In [103]:
rec_games(167355, info, X, k=10)

Because you enjoyed playing: nemesis
I think you would enjoy these:
lords of hellas
rising sun
blood rage
mansions of madness: second edition
scythe
terraforming mars
root
star wars: rebellion
tainted grail: the fall of avalon


### Recommending games to my friends

In [104]:
info[info.title.str.contains('clank')]

,game_id,title
7980,201808,clank!: a deck-building adventure
6544255,233371,clank! in! space!: a deck-building adventure
11027330,266507,clank! legacy: acquisitions incorporated


In [105]:
rec_games(266507, info, X, k=10)

Because you enjoyed playing: clank! legacy: acquisitions incorporated
I think you would enjoy these:
clank! in! space!: a deck-building adventure
clank!: a deck-building adventure
the quacks of quedlinburg
space base
everdell
roll player
wingspan
tapestry
charterstone


In [106]:
info[info.title.str.contains('everrain')]

,game_id,title
15797255,252315,the everrain


In [107]:
rec_games(252315, info, X, k=10)

Because you enjoyed playing: the everrain
I think you would enjoy these:
oathsworn: into the deepwood
solomon kane
warhammer underworlds: dreadfane
limbo: eternal war
mighty morphin power rangers game
waste knights: second edition
the great wall
krosmaster: blast
village attacks


In [108]:
info[info.title.str.contains('twilight')]

,game_id,title
2793,12333,twilight struggle
3658174,12493,twilight imperium (third edition)
5359694,233078,twilight imperium (fourth edition)
13318833,180199,"colonial twilight: the french-algerian war, 19..."
13383735,24,twilight imperium
13756126,26055,twilight imperium (second edition)
14506148,191364,twilight squabble
14844496,63385,anima: twilight of the gods
14932425,21779,1914: twilight in the east
15298572,206904,twilight of the gods


In [109]:
rec_games(233078, info, X, k=10)

Because you enjoyed playing: twilight imperium (fourth edition)
I think you would enjoy these:
star wars: rebellion
scythe
terraforming mars
blood rage
rising sun
root
twilight imperium (third edition)
gloomhaven
through the ages: a new story of civilization


In [110]:
info[info.title.str.contains('flatline')]

,game_id,title
12481842,216597,flatline


In [111]:
rec_games(216597, info, X, k=10)

Because you enjoyed playing: flatline
I think you would enjoy these:
fuse
flip ships
sentient
covert
ex libris
pandemic: the cure
magic maze
quadropolis
potion explosion


In [112]:
info[info.title.str.contains('terraforming')]

,game_id,title
1000,167791,terraforming mars


In [114]:
rec_games(167791, info, X, k=10)

Because you enjoyed playing: terraforming mars
I think you would enjoy these:
scythe
great western trail
7 wonders duel
azul
wingspan
codenames
the castles of burgundy
7 wonders
viticulture essential edition
